In [1]:
import twitter
from os.path import isfile
import pandas as pd
from time import sleep

In [2]:
api = twitter.Api(consumer_key='foYe2yl6efwHz9kJQiyI7vuSp',
                  consumer_secret="fITQUjsbsC938lb5VQpzFaCuW6d3LXhC5e34AkCtL5xkIvj1BD",
                  access_token_key='1110034001052471298-Bo7P9PV9bTlzIfxDT00DvW2TB4PsB8',
                  access_token_secret='JXh4LO37BLVqyiAhrXvwlh8riHe9EhTEnpqtAbXejz1WR')
api.InitializeRateLimit()

In [8]:
def twitter_search(title):
    ratelim = twitter.ratelimit.RateLimit()
    remaining = ratelim.get_limit("https://api.twitter.com/1.1/application/rate_limit_status.json").remaining
    print(f'remaining = {remaining}')

    if remaining <= 0:
        print("failed")
        return
    
    sub_dict = {'game_title': [], 'post_title': [], 'score': []}
    csv = f'game_titles_twitter.csv'

    # Set csv_loaded to True if csv exists since you can't evaluate the
    # truth value of a DataFrame.
    df, csv_loaded = (pd.read_csv(csv), 1) if isfile(csv) else ('', 0)

    #print(f'csv = {csv}')
    #print(f'csv_loaded = {csv_loaded}')
    print(f'Collecting information from Twitter for search term "{title}".')
    
    search = "q=" + title.replace(" ", "%20")
    #print(search)
    feed = api.GetSearch(raw_query=search)
    
    for post in feed:
        if post.retweeted_status is not None:
            notDuplicated = post.retweeted_status.text not in sub_dict['post_title']
            notCopied = post.retweeted_status.text not in tuple(df.post_title) if csv_loaded else True
            if notDuplicated and notCopied:
                #print(post.retweeted_status.favorite_count)
                # Save any unique, non-stickied posts with descriptions to sub_dict.
                sub_dict['game_title'].append(title)
                sub_dict['post_title'].append(post.retweeted_status.text)
                sub_dict['score'].append(post.retweeted_status.favorite_count)
                sleep(0.1)
                
    sub_dict_final = {'game_title': [title], 'score': [sum(sub_dict['score'])]}

    # print(sub_dict)
    new_df = pd.DataFrame(sub_dict_final)

    # Add new_df to df if df exists then save it to a csv.
    if 'DataFrame' in str(type(df)):
        pd.concat([df, new_df], axis=0, sort=0).to_csv(csv, index=False)
        #print(f'{len(new_df)} new posts collected and added to {csv}')
    else:
        new_df.to_csv(csv, index=False)
        #print(f'{len(new_df)} posts collected and saved to {csv}')

In [10]:
#twitter_search("Overwatch")

remaining = 15


In [5]:
#results = api.GetSearch(raw_query="q=twitter%20&result_type=recent&since=2014-07-19&count=100")

In [6]:
#result = results[0]
#print(result)

In [7]:
#ratelim = twitter.ratelimit.RateLimit()
#ratelim.get_limit("https://api.twitter.com/1.1/application/rate_limit_status.json")